In [73]:
import psycopg2
from psycopg2 import extras
import sys, io
import sqlite3
import seaborn as sns
import pandas as pd

sys.path.append("..")
import postgres_config

io.DEFAULT_BUFFER_SIZE = 8192*4
DB_TABLE = "gdelt_raw"
CONNECTION_DETAILS = (
    f"host={postgres_config.HOST} "
    "dbname=gdelt "
    "user=postgres "
    f"password={postgres_config.PASSWORD}"
)

In [165]:
def get_polarity_in_db():
    """Returns list of datetime and polarity in the database.
    """
    with psycopg2.connect(CONNECTION_DETAILS) as conn:
        with conn.cursor() as cur:
            try:
                df_polarity = pd.read_sql(
                    "SELECT datetime,countries,polarity FROM gdelt_raw WHERE DATE(datetime) > DATE('2020-01-01 00:00:00');",
                    conn,
                    parse_dates=["datetime"])
            except Exception as e:
                print(f"Exception executing Select Query: {e}")
                print(f"Exception type: {type(e)}")
    
    return df_polarity

In [166]:
df_polarity = get_polarity_in_db()
print(df_polarity)

                    datetime         countries  polarity
0        2021-08-02 15:45:00              [AS]  4.413793
1        2021-08-02 15:45:00              [CA]  4.911243
2        2021-08-02 15:45:00          [AS, UK]  6.802721
3        2021-08-02 15:45:00              [AS]  3.189793
4        2021-08-02 15:45:00              [UK]  2.970297
...                      ...               ...       ...
13404946 2021-08-02 15:45:00          [NZ, UK]  2.884615
13404947 2021-08-02 15:45:00              [AS]  6.757913
13404948 2021-08-02 15:45:00              [UK]  7.339450
13404949 2021-08-02 15:45:00  [AS, UK, CA, NZ]  2.290076
13404950 2021-08-02 14:45:00              [UK]  5.586592

[13404951 rows x 3 columns]


In [167]:
df_polarity_long = [] 
for index,row in df_polarity.iterrows() :
    if len(row['countries'])>1:
        for country in row['countries']: 
            df_polarity_long.append([row['datetime'].date(),country,row['polarity']])
    else :
        df_polarity_long.append([row['datetime'].date(),row['countries'].pop(),row['polarity']])

df_polarity_long = pd.DataFrame(data=df_polarity_long,columns=['datetime','country','polarity'])
print(df_polarity_long)

            datetime country  polarity
0         2021-08-02      AS  4.413793
1         2021-08-02      CA  4.911243
2         2021-08-02      AS  6.802721
3         2021-08-02      UK  6.802721
4         2021-08-02      AS  3.189793
...              ...     ...       ...
16442661  2021-08-02      AS  2.290076
16442662  2021-08-02      UK  2.290076
16442663  2021-08-02      CA  2.290076
16442664  2021-08-02      NZ  2.290076
16442665  2021-08-02      UK  5.586592

[16442666 rows x 3 columns]


In [ ]:
polarity_agg = df_polarity_long.groupby(['datetime','country'])['polarity'].rolling(window=7)

sns.lineplot(x="datetime", y="polarity", hue='country', data=pd.DataFrame(polarity_agg))